In [15]:
import pandas as pd

data_path = "../data/raw/bf_efficiency_raw_data.csv"
df = pd.read_csv(data_path)

In [16]:
df = pd.read_csv(data_path)
df.shape

(140, 25)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   line_id              140 non-null    object
 1   trial_id             140 non-null    object
 2   colony_strength      140 non-null    object
 3   BF1                  140 non-null    object
 4   BF2                  140 non-null    object
 5   BF3                  140 non-null    object
 6   BF4                  140 non-null    object
 7   bf1_amounts_ml       140 non-null    int64 
 8   bf2_amounts_ml       140 non-null    int64 
 9   bf3_amounts_ml       140 non-null    int64 
 10  bf4_amounts_ml       140 non-null    int64 
 11  collection_1_date    140 non-null    object
 12  collection_2_date    140 non-null    object
 13  collection_3_date    140 non-null    object
 14  collection_4_date    140 non-null    object
 15  collection_1_output  140 non-null    object
 16  collecti

In [18]:
numeric_cols = [
    "collection_1_output",
    "collection_2_output",
    "collection_3_output",
    "collection_4_output",
    "blood_input_ml",
    "egg_output_total"
]

for col in numeric_cols:
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(",", "", regex=False)
        .astype(float)
    )

df[numeric_cols].dtypes

collection_1_output    float64
collection_2_output    float64
collection_3_output    float64
collection_4_output    float64
blood_input_ml         float64
egg_output_total       float64
dtype: object

In [19]:
date_cols = [
    "collection_1_date",
    "collection_2_date",
    "collection_3_date",
    "collection_4_date"
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col], dayfirst=True, errors="coerce")

df[date_cols].head()

,collection_1_date,collection_2_date,collection_3_date,collection_4_date
0,2025-09-03,2025-09-05,2025-09-09,2025-09-12
1,2025-09-03,2025-09-05,2025-09-09,2025-09-12
2,2025-09-03,2025-09-05,2025-09-09,2025-09-12
3,2025-09-03,2025-09-05,2025-09-09,2025-09-12
4,2025-09-03,2025-09-05,2025-09-09,2025-09-12


In [21]:
df["efficiency"] = df["egg_output_total"] / df["blood_input_ml"]
df["eggs_per_hour"] = df["egg_output_total"] / df["time_between_bf_hrs"]

df[["blood_input_ml", "egg_output_total", "efficiency", "eggs_per_hour"]].head()


,blood_input_ml,egg_output_total,efficiency,eggs_per_hour
0,2400.0,1065500.0,443.958333,22197.916667
1,2400.0,1155000.0,481.250000,24062.500000
2,2400.0,1917000.0,798.750000,39937.500000
3,2400.0,1136000.0,473.333333,23666.666667
4,2400.0,1750000.0,729.166667,36458.333333


In [22]:
df.shape

(140, 27)

In [23]:
df.describe()


,bf1_amounts_ml,bf2_amounts_ml,bf3_amounts_ml,bf4_amounts_ml,collection_1_date,collection_2_date,collection_3_date,collection_4_date,collection_1_output,collection_2_output,collection_3_output,collection_4_output,time_between_bf_hrs,blood_input_ml,egg_output_total,humidity_pct,temperature_c,efficiency,eggs_per_hour
count,140.000000,140.000000,140.000000,140.000000,140,140,140,140,1.400000e+02,1.400000e+02,1.400000e+02,140.000000,140.0,140.000000,1.400000e+02,140.0,140.0,140.000000,140.000000
mean,599.428571,599.428571,599.428571,599.428571,2025-10-09 18:00:00,2025-10-11 18:00:00,2025-10-15 18:00:00,2025-10-18 18:00:00,5.053286e+05,5.088286e+05,3.769893e+05,350740.714286,48.0,2397.714286,1.741887e+06,75.0,27.0,719.449977,36289.315476
min,60.000000,60.000000,60.000000,60.000000,2025-09-03 00:00:00,2025-09-05 00:00:00,2025-09-09 00:00:00,2025-09-12 00:00:00,2.000000e+03,2.500000e+03,2.500000e+03,2700.000000,48.0,240.000000,9.700000e+03,75.0,27.0,40.416667,202.083333
25%,600.000000,600.000000,600.000000,600.000000,2025-09-17 00:00:00,2025-09-19 00:00:00,2025-09-23 00:00:00,2025-09-26 00:00:00,3.700000e+05,3.792500e+05,2.695000e+05,242250.000000,48.0,2400.000000,1.317000e+06,75.0,27.0,549.375000,27437.500000
50%,600.000000,600.000000,600.000000,600.000000,2025-10-08 00:00:00,2025-10-10 00:00:00,2025-10-14 00:00:00,2025-10-17 00:00:00,4.900000e+05,5.200000e+05,3.710000e+05,325000.000000,48.0,2400.000000,1.754000e+06,75.0,27.0,696.250000,36541.666667
75%,600.000000,600.000000,600.000000,600.000000,2025-10-29 00:00:00,2025-10-31 00:00:00,2025-11-04 00:00:00,2025-11-07 00:00:00,6.000000e+05,6.300000e+05,4.477500e+05,446750.000000,48.0,2400.000000,2.081250e+06,75.0,27.0,860.416667,43359.375000
max,800.000000,800.000000,800.000000,800.000000,2025-11-19 00:00:00,2025-11-21 00:00:00,2025-11-25 00:00:00,2025-11-28 00:00:00,1.340000e+06,1.240000e+06,1.065000e+06,775000.000000,48.0,3200.000000,4.355000e+06,75.0,27.0,1814.583333,90729.166667
std,103.435737,103.435737,103.435737,103.435737,NaN,NaN,NaN,NaN,2.393451e+05,2.152728e+05,1.782216e+05,158431.917564,0.0,413.742948,6.763886e+05,0.0,0.0,272.208005,14091.428440


In [24]:
output_path = "../data/processed/bf_efficiency_clean.csv"
df.to_csv(output_path, index=False)